In [ ]:
import numpy as np

def matmul_mod(a, b, m):
    return np.remainder(np.dot(a, b), m)

def pow_mat_mod(A, e, m, err_sq=None):
    R = np.identity(2, dtype=object)
    B = A.copy()
    mul_id = sq_id = 0
    while e:
        if e & 1:
            R = matmul_mod(R, B, m)
            mul_id += 1
        if err_sq is not None and sq_id == err_sq:
            B[0,0] = (B[0,0] + 123456789) % m
        B = matmul_mod(B, B, m)
        sq_id += 1
        e >>= 1
    return R

def gerbicz_check(A, power, m, r):
    q = power // r - 1
    X  = pow_mat_mod(A, r,         m)
    Y2 = pow_mat_mod(X, q + 1,     m)
    Y1 = pow_mat_mod(A, power,     m)
    ok = np.all(np.remainder(Y1 - Y2, m) == 0)
    return ok, Y1

def lucas_lehmer_mod(p, err_sq=None):
    Mp = (1 << p) - 1
    A  = np.array([[4, -1],[1, 0]], dtype=object)
    init = np.array([[4],[2]], dtype=object)
    power = 1 << (p - 2)
    r = 1 << ((p - 2)//2)

    ok, Apow_ref = gerbicz_check(A, power, Mp, r)
    if not ok:
        raise ValueError("Gerbicz reference failed")

    Apow_run = pow_mat_mod(A, power, Mp, err_sq=err_sq)
    if not np.all(np.remainder(Apow_run - Apow_ref, Mp) == 0):
        raise ValueError("Gerbicz run failed")

    v = matmul_mod(Apow_run, init, Mp)
    s = int(v[1,0])
    print(f"s_{p-2} mod M_{p} = {s}")
    print(f"2^{p}-1 = {Mp} is {'prime' if s==0 else 'composite'}")

lucas_lehmer_mod(13)

try:
    lucas_lehmer_mod(13, err_sq=3)
except ValueError as e:
    print(e)


try:
    lucas_lehmer_mod(13, err_sq=13)
except ValueError as e:
    print(e)

s_11 mod M_13 = 0
2^13-1 = 8191 is prime
Gerbicz run failed
s_11 mod M_13 = 0
2^13-1 = 8191 is prime


In [ ]:
def run_case(p, err_sq, should_raise):
    try:
        lucas_lehmer_mod(p, err_sq=err_sq)
    except ValueError:
        if not should_raise:
            print("FAIL", p, err_sq, "unexpected error")
        else:
            print("OK", p, err_sq, "error detected")
        return
    if should_raise:
        print("FAIL", p, err_sq, "error NOT detected")
    else:
        print("OK", p, err_sq, "no error")

cases = [
    (13, None,  False),
    (13, 0,     True),
    (13, 1,     True),
    (13, 2,     True),
    (13, 3,     True),
    (13, 4,     True),
    (13, 5,     True),
    (13, 6,     True),
    (13, 7,     True),
    (13, 8,     True),
    (13, 9,     True),
    (13, 10,    True),
    (11, None,  False),
    (11, 0,     True),
    (11, 1,     True),
    (11, 2,     True),
    (11, 3,     True),
    (11, 4,     True),
    (11, 5,     True),
    (11, 6,     True),
    (17, None,  False),
    (17, 0,     True),
    (17, 1,     True),
    (17, 2,     True),
    (17, 3,     True),
    (17, 4,     True),
    (17, 5,     True),
    (17, 6,     True),
    (17, 7,     True),
    (17, 8,     True),
    (17, 9,     True),
    (17, 10,    True),
    (17, 11,    True),
    (17, 12,    True),
]

for p, err_sq, should_raise in cases:
    run_case(p, err_sq, should_raise)


s_11 mod M_13 = 0
2^13-1 = 8191 is prime
OK 13 None no error
OK 13 0 error detected
OK 13 1 error detected
OK 13 2 error detected
OK 13 3 error detected
OK 13 4 error detected
OK 13 5 error detected
OK 13 6 error detected
OK 13 7 error detected
OK 13 8 error detected
OK 13 9 error detected
OK 13 10 error detected
s_9 mod M_11 = 1736
2^11-1 = 2047 is composite
OK 11 None no error
OK 11 0 error detected
OK 11 1 error detected
OK 11 2 error detected
OK 11 3 error detected
OK 11 4 error detected
OK 11 5 error detected
OK 11 6 error detected
s_15 mod M_17 = 0
2^17-1 = 131071 is prime
OK 17 None no error
OK 17 0 error detected
OK 17 1 error detected
OK 17 2 error detected
OK 17 3 error detected
OK 17 4 error detected
OK 17 5 error detected
OK 17 6 error detected
OK 17 7 error detected
OK 17 8 error detected
OK 17 9 error detected
OK 17 10 error detected
OK 17 11 error detected
OK 17 12 error detected


In [1]:
def add_mod(a,b,m): return (a+b)%m
def sub_mod(a,b,m): return (a-b)%m
def mul_mod(a,b,m): return (a*b)%m
def sqr_mod(a,m):   return (a*a)%m

def matmul2_mod(A,B,m):
    return (
        add_mod(mul_mod(A[0],B[0],m), mul_mod(A[1],B[2],m), m),
        add_mod(mul_mod(A[0],B[1],m), mul_mod(A[1],B[3],m), m),
        add_mod(mul_mod(A[2],B[0],m), mul_mod(A[3],B[2],m), m),
        add_mod(mul_mod(A[2],B[1],m), mul_mod(A[3],B[3],m), m)
    )

def matpow2_block(A, blk, m):
    R = (1,0,0,1)
    B = A
    e = blk
    while e:
        if e&1: R = matmul2_mod(R,B,m)
        B = matmul2_mod(B,B,m)
        e >>= 1
    return R

def vecmul2_mod(A,v,m):
    return (
        add_mod(mul_mod(A[0],v[0],m), mul_mod(A[1],v[1],m), m),
        add_mod(mul_mod(A[2],v[0],m), mul_mod(A[3],v[1],m), m)
    )

def lucas_lehmer_mod_online(p, err_blk=None):
    m = (1<<p)-1
    A = (4, (-1)%m, 1, 0)
    init = (4%m, 2%m)
    e = 1<<(p-2)

    r = 1<<((p-2)//2)
    q = e//r - 1

    R = (1,0,0,1)
    X = None
    Y2 = (1,0,0,1)
    blk_id = 0
    left = e

    while left:
        blk = min(r, left)
        B = matpow2_block(A, blk, m)
        if err_blk is not None and blk_id == err_blk:
            B = (add_mod(B[0],123456789,m),B[1],B[2],B[3])
        R = matmul2_mod(R,B,m)
        if X is None:
            X = B
        else:
            Y2 = matmul2_mod(Y2,X,m)
        blk_id += 1
        left -= blk
        if blk_id == q+1:
            if not all((R[i]-matmul2_mod(Y2,X,m)[i])%m==0 for i in range(4)):
                raise ValueError("Gerbicz failed")
            Y2 = (1,0,0,1)
            blk_id = 0
            X = None

    v = vecmul2_mod(R, init, m)
    s = v[1]
    print(f"s_{p-2} mod M_{p} = {s}")
    print(f"2^{p}-1 = {m} is {'prime' if s==0 else 'composite'}")

lucas_lehmer_mod_online(13)
try: lucas_lehmer_mod_online(13, err_blk=2)
except ValueError as e: print(e)


s_11 mod M_13 = 0
2^13-1 = 8191 is prime
Gerbicz failed


In [4]:
def run_case(p, err_sq, should_raise):
    try:
        lucas_lehmer_mod_online(p, err_blk=err_sq)
    except ValueError:
        if not should_raise:
            print("FAIL", p, err_sq, "unexpected error")
        else:
            print("OK", p, err_sq, "error detected")
        return
    if should_raise:
        print("FAIL", p, err_sq, "error NOT detected")
    else:
        print("OK", p, err_sq, "no error")

cases = [
    (13, None,  False),
    (13, 0,     True),
    (13, 1,     True),
    (13, 2,     True),
    (13, 3,     True),
    (13, 4,     True),
    (13, 5,     True),
    (13, 6,     True),
    (13, 7,     True),
    (13, 8,     True),
    (13, 9,     True),
    (13, 10,    True),
    (11, None,  False),
    (11, 0,     True),
    (11, 1,     True),
    (11, 2,     True),
    (11, 3,     True),
    (11, 4,     True),
    (11, 5,     True),
    (11, 6,     True),
    (17, None,  False),
    (17, 0,     True),
    (17, 1,     True),
    (17, 2,     True),
    (17, 3,     True),
    (17, 4,     True),
    (17, 5,     True),
    (17, 6,     True),
    (17, 7,     True),
    (17, 8,     True),
    (17, 9,     True),
    (17, 10,    True),
    (17, 11,    True),
    (17, 12,    True),
]

for p, err_sq, should_raise in cases:
    run_case(p, err_sq, should_raise)


s_11 mod M_13 = 0
2^13-1 = 8191 is prime
OK 13 None no error
OK 13 0 error detected
OK 13 1 error detected
OK 13 2 error detected
OK 13 3 error detected
OK 13 4 error detected
OK 13 5 error detected
OK 13 6 error detected
OK 13 7 error detected
OK 13 8 error detected
OK 13 9 error detected
OK 13 10 error detected
s_9 mod M_11 = 1736
2^11-1 = 2047 is composite
OK 11 None no error
OK 11 0 error detected
OK 11 1 error detected
OK 11 2 error detected
OK 11 3 error detected
OK 11 4 error detected
OK 11 5 error detected
OK 11 6 error detected
s_15 mod M_17 = 0
2^17-1 = 131071 is prime
OK 17 None no error
OK 17 0 error detected
OK 17 1 error detected
OK 17 2 error detected
OK 17 3 error detected
OK 17 4 error detected
OK 17 5 error detected
OK 17 6 error detected
OK 17 7 error detected
OK 17 8 error detected
OK 17 9 error detected
OK 17 10 error detected
OK 17 11 error detected
OK 17 12 error detected


In [35]:
def addm(a,b,m): return (a+b)%m
def subm(a,b,m): return (a-b)%m
def sqrm(a,m):   return (a*a)%m
def step(x,m):   return subm(sqrm(x,m),2,m)

def run_block(x,cnt,m,err_at=None,off=0):
    for i in range(cnt):
        if err_at is not None and off+i==err_at:
            x = addm(sqrm(x,m),123456789,m)
        else:
            x = step(x,m)
    return x

def ll_gec_online(p,r=None,err_iter=None):
    m = (1<<p)-1
    n = p-2
    if r is None or r<=0 or r>n: r = max(1,int(n**0.5))
    s = 4
    done = 0
    while True:
        rem = n-done
        if rem < 2*r:
            s = run_block(s, rem, m, err_iter, done)
            break
        q = rem//r - 1
        win = r*(q+1)
        z0 = s
        z  = z0
        off = done
        X = None
        for j in range(q+1):
            z = run_block(z, r, m, err_iter, off)
            if j==0: X = z
            off += r
        z_err = z
        Xc = run_block(z0, r, m)
        Yc = Xc
        for _ in range(q):
            Yc = run_block(Yc, r, m)
        if z_err != Yc:
            raise ValueError("Gerbicz failed")
        s = z_err
        done += win
    print(f"s_{n} mod M_{p} = {s}")
    print(f"2^{p}-1 = {m} is {'prime' if s==0 else 'composite'}")

ll_gec_online(89)
try: ll_gec_online(89, err_iter=9)
except ValueError as e: print(e)


s_87 mod M_89 = 0
2^89-1 = 618970019642690137449562111 is prime
Gerbicz failed
